In [2]:
import numpy as np
import pandas as pd
from xgboost import XGBClassifier
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix
import os

/home/phiha/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
data_features = pd.read_csv(os.path.join('data', 'NUSW-NB15_features.csv'), header=None).values
features = data_features[1:, :]
feature_names = features[:, 1]  # Names of the features in a list
feature_types = np.array(
    [item.lower() for item in features[:-2, 2]])  # The types of the corresponding features in 'features_names'

In [4]:
nominal_cols = np.where(feature_types == "nominal")[0]
integer_cols = np.where(feature_types == "integer")[0]
float_cols = np.where(feature_types == "float")[0]
binary_cols = np.where(feature_types == "binary")[0]

In [5]:
dataframe1 = pd.read_csv(os.path.join("data","UNSW-NB15_1.csv"), header=None)
dataframe2 = pd.read_csv(os.path.join("data","UNSW-NB15_2.csv"), header=None)
dataframe3 = pd.read_csv(os.path.join("data","UNSW-NB15_3.csv"), header=None)
dataframe4 = pd.read_csv(os.path.join("data","UNSW-NB15_4.csv"), header=None)

/home/phiha/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1,3,47) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/phiha/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (3,39,47) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
print('Concatenating...')
dataframe = pd.concat([dataframe1, dataframe2, dataframe3, dataframe4])

del dataframe1
del dataframe2
del dataframe3
del dataframe4

Concatenating...


In [7]:
# dataframe.columns = feature_names

In [9]:
#dataframe.service.value_counts()

In [10]:
dataframe.columns[47]

47

In [11]:
dataframe.iloc[:,13].unique()

array(['dns', '-', 'http', 'smtp', 'ftp-data', 'ftp', 'ssh', 'pop3',
       'snmp', 'ssl', 'irc', 'radius', 'dhcp', nan], dtype=object)

In [12]:
dataframe[47].fillna("normal", inplace=True)


In [13]:
df2 = dataframe.iloc[-10:, 10:20]

In [14]:
df2

,10,11,12,13,14,15,16,17,18,19
440034,29.0,2.0,57.0,-,12496.513670,5.983751e+05,58.0,116.0,255.0,255.0
440035,29.0,2.0,21.0,-,7687.937988,2.913199e+05,22.0,44.0,255.0,255.0
440036,29.0,2.0,262.0,http,198981.250000,1.091598e+07,262.0,526.0,255.0,255.0
440037,29.0,5.0,6.0,ftp,2808.172607,3.428493e+03,20.0,20.0,255.0,255.0
440038,29.0,2.0,262.0,-,198981.250000,1.091598e+07,262.0,526.0,255.0,255.0
440039,29.0,1.0,2.0,ftp-data,24465.671880,1.466108e+05,6.0,8.0,255.0,255.0
440040,29.0,2.0,2.0,ftp,8743.816406,6.333240e+03,8.0,6.0,255.0,255.0
440041,29.0,7.0,9.0,ftp,2204.839844,2.549583e+03,32.0,30.0,255.0,255.0
440042,29.0,2.0,57.0,http,12496.513670,5.983751e+05,58.0,116.0,255.0,255.0
440043,252.0,5.0,6.0,pop3,4470.913574,5.259898e+03,12.0,12.0,255.0,255.0


In [15]:
attact_cat = dataframe.iloc[:, 47]
labels = dataframe.iloc[:, 48]
labels.fillna(0, inplace=True)
dataframe = dataframe.iloc[:, :-2]

In [16]:
df = pd.get_dummies(dataframe, columns=nominal_cols[2:])
df.replace(r'\s+', np.nan, regex=True, inplace=True)
df.drop(df.columns[[0,1,2,3]], axis=1, inplace=True)
df.fillna(0, inplace=True)

In [17]:
df.iloc[-10:,191:]

,13_-,13_dhcp,13_dns,13_ftp,13_ftp-data,13_http,13_irc,13_pop3,13_radius,13_smtp,13_snmp,13_ssh,13_ssl
440034,1,0,0,0,0,0,0,0,0,0,0,0,0
440035,1,0,0,0,0,0,0,0,0,0,0,0,0
440036,0,0,0,0,0,1,0,0,0,0,0,0,0
440037,0,0,0,1,0,0,0,0,0,0,0,0,0
440038,1,0,0,0,0,0,0,0,0,0,0,0,0
440039,0,0,0,0,1,0,0,0,0,0,0,0,0
440040,0,0,0,1,0,0,0,0,0,0,0,0,0
440041,0,0,0,1,0,0,0,0,0,0,0,0,0
440042,0,0,0,0,0,1,0,0,0,0,0,0,0
440043,0,0,0,0,0,0,0,1,0,0,0,0,0


In [18]:
df1 = df.iloc[-10:,191:]

In [19]:
import re
df1.rename(columns=lambda x: re.sub('13','srv',x))

,srv_-,srv_dhcp,srv_dns,srv_ftp,srv_ftp-data,srv_http,srv_irc,srv_pop3,srv_radius,srv_smtp,srv_snmp,srv_ssh,srv_ssl
440034,1,0,0,0,0,0,0,0,0,0,0,0,0
440035,1,0,0,0,0,0,0,0,0,0,0,0,0
440036,0,0,0,0,0,1,0,0,0,0,0,0,0
440037,0,0,0,1,0,0,0,0,0,0,0,0,0
440038,1,0,0,0,0,0,0,0,0,0,0,0,0
440039,0,0,0,0,1,0,0,0,0,0,0,0,0
440040,0,0,0,1,0,0,0,0,0,0,0,0,0
440041,0,0,0,1,0,0,0,0,0,0,0,0,0
440042,0,0,0,0,0,1,0,0,0,0,0,0,0
440043,0,0,0,0,0,0,0,1,0,0,0,0,0


In [20]:
import time

In [27]:
X = df.astype(np.float64)

In [28]:
X.shape

(2086976, 204)

In [29]:
y = labels.astype(np.float64)

In [30]:
y.shape

(2086976,)

In [31]:
seed = 125
test_size = 0.2
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed, stratify=y)
# fit model no training data


In [42]:
#BAGGING TREE
from sklearn.ensemble import BaggingClassifier
model = BaggingClassifier()
start = time.time()
model.fit(X_train, y_train)
end = time.time()
print 'Training in %.2f'%(end-start)
y_pred = model.predict(X_test)
print confusion_matrix(y_true=y_test, y_pred=y_pred)

Training in 138.07
[[352173    966]
 [  1500  62757]]


In [43]:
#RF
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
start = time.time()
model.fit(X_train, y_train)
end = time.time()
print 'Training in %.2f'%(end-start)
y_pred = model.predict(X_test)
print confusion_matrix(y_true=y_test, y_pred=y_pred)

Training in 17.87
[[352045   1094]
 [  1680  62577]]


In [36]:
model = XGBClassifier()
start = time.time()
model.fit(X_train, y_train)
end = time.time()
print 'Training in %.2f'%(end-start)

Training in 66.68


In [37]:
y_pred = model.predict(X_test)

In [38]:
print confusion_matrix(y_true=y_test, y_pred=y_pred)

[[351325   1814]
 [  2112  62145]]


In [39]:
from sklearn.metrics import classification_report


In [40]:
print(classification_report(y_pred,y_test))

             precision    recall  f1-score   support

        0.0       0.99      0.99      0.99    353437
        1.0       0.97      0.97      0.97     63959

avg / total       0.99      0.99      0.99    417396



In [45]:
attact_cat.unique()

array(['normal', 'Exploits', 'Reconnaissance', 'DoS', 'Generic',
       'Shellcode', ' Fuzzers', 'Worms', 'Backdoors', 'Analysis',
       ' Reconnaissance ', 'Backdoor', ' Fuzzers ', ' Shellcode '], dtype=object)

In [47]:
from sklearn.multiclass import OneVsRestClassifier
# from sklearn.model_selection import trai

In [57]:
y = attact_cat.values
seed = 125
test_size = 0.2
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed, stratify=y)

In [65]:
np.unique(y_test)

array([' Fuzzers', ' Fuzzers ', ' Reconnaissance ', ' Shellcode ',
       'Analysis', 'Backdoor', 'Backdoors', 'DoS', 'Exploits', 'Generic',
       'Reconnaissance', 'Shellcode', 'Worms', 'normal'], dtype=object)

In [61]:
#BAGGING TREE
from sklearn.ensemble import BaggingClassifier
model = BaggingClassifier(verbose=True, n_jobs=-1)
start = time.time()
model.fit(X_train, y_train)
print model
end = time.time()
print 'Training in %.2f'%(end-start)
y_pred = model.predict(X_test)
#print confusion_matrix(y_true=y_test, y_pred=y_pred)
print classification_report(y_pred,y_test)

[Parallel(n_jobs=10)]: Done   2 out of  10 | elapsed:   28.3s remaining:  1.9min
[Parallel(n_jobs=10)]: Done  10 out of  10 | elapsed:  1.1min finished


BaggingClassifier(base_estimator=None, bootstrap=True,
         bootstrap_features=False, max_features=1.0, max_samples=1.0,
         n_estimators=10, n_jobs=-1, oob_score=False, random_state=None,
         verbose=True, warm_start=False)
Training in 66.26


[Parallel(n_jobs=10)]: Done   2 out of  10 | elapsed:    3.4s remaining:   13.6s
[Parallel(n_jobs=10)]: Done  10 out of  10 | elapsed:   13.0s finished


                  precision    recall  f1-score   support

         Fuzzers       0.76      0.63      0.69      1217
        Fuzzers        0.68      0.71      0.69      3657
 Reconnaissance        0.77      0.90      0.83      2095
      Shellcode        0.65      0.67      0.66       250
        Analysis       0.12      0.59      0.20       106
        Backdoor       0.13      0.92      0.22        50
       Backdoors       0.02      0.17      0.03        12
             DoS       0.28      0.33      0.30      2777
        Exploits       0.79      0.63      0.70     11183
         Generic       0.99      1.00      0.99     42628
  Reconnaissance       0.88      0.92      0.90       339
       Shellcode       0.50      0.58      0.54        38
           Worms       0.46      0.67      0.54        24
          normal       1.00      1.00      1.00    353020

     avg / total       0.98      0.98      0.98    417396



In [62]:
#RF
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(verbose=True, n_jobs=-1)
print model
start = time.time()
model.fit(X_train, y_train)
end = time.time()
print 'Training in %.2f'%(end-start)
y_pred = model.predict(X_test)
#print confusion_matrix(y_true=y_test, y_pred=y_pred)
print classification_report(y_pred,y_test)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
            oob_score=False, random_state=None, verbose=True,
            warm_start=False)


[Parallel(n_jobs=-1)]: Done   8 out of  10 | elapsed:    4.3s remaining:    1.1s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    4.6s finished


Training in 7.32


[Parallel(n_jobs=10)]: Done   2 out of  10 | elapsed:    0.2s remaining:    0.9s
[Parallel(n_jobs=10)]: Done  10 out of  10 | elapsed:    0.3s finished


                  precision    recall  f1-score   support

         Fuzzers       0.71      0.59      0.64      1213
        Fuzzers        0.61      0.67      0.64      3498
 Reconnaissance        0.76      0.90      0.83      2070
      Shellcode        0.53      0.57      0.55       244
        Analysis       0.09      0.59      0.16        83
        Backdoor       0.11      0.72      0.18        53
       Backdoors       0.01      0.05      0.02        20
             DoS       0.24      0.31      0.27      2609
        Exploits       0.80      0.62      0.70     11410
         Generic       0.98      1.00      0.99     42491
  Reconnaissance       0.87      0.91      0.89       336
       Shellcode       0.41      0.62      0.49        29
           Worms       0.09      0.50      0.15         6
          normal       1.00      1.00      1.00    353334

     avg / total       0.98      0.98      0.98    417396



In [72]:
#RF
from sklearn.ensemble import AdaBoostClassifier
model = AdaBoostClassifier()
print model
start = time.time()
model.fit(X_train, y_train)
end = time.time()
print 'Training in %.2f'%(end-start)
y_pred = model.predict(X_test)
#print confusion_matrix(y_true=y_test, y_pred=y_pred)
print classification_report(y_pred,y_test)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=50, random_state=None)
Training in 441.04
                  precision    recall  f1-score   support

         Fuzzers       0.08      0.56      0.14       140
        Fuzzers        0.00      0.00      0.00         0
 Reconnaissance        0.00      0.00      0.00         0
      Shellcode        0.00      0.00      0.00         0
        Analysis       0.00      0.00      0.00       576
        Backdoor       0.00      0.00      0.00         0
       Backdoors       0.00      0.00      0.00         0
             DoS       0.01      0.34      0.01        59
        Exploits       0.84      0.51      0.64     14507
         Generic       0.97      0.99      0.98     42201
  Reconnaissance       0.00      0.00      0.00         0
       Shellcode       0.00      0.00      0.00         0
           Worms       0.11      0.00      0.00      1796
          normal       0.99      0.98     

In [69]:
#XGBoost
model = XGBClassifier(max_depth=3, n_estimators=10)
start = time.time()
model.fit(X_train, y_train)
print model
end = time.time()
print 'Training in %.2f'%(end-start)
y_pred = model.predict(X_test)
print confusion_matrix(y_true=y_test, y_pred=y_pred)
print classification_report(y_pred,y_test)

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=10, nthread=-1,
       objective='multi:softprob', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)
Training in 82.24
[[   246      0      5      0      0      0      0      0    103      0
      14      0      0    642]
 [     0    188     89      0      0      0      0    145    451     16
       0      0      0   2950]
 [     0     28   1460      0      0      0      0    164    497      0
       0      0      0    297]
 [     0      3    127      0      0      0      0      0     56      0
       0      0      0     72]
 [    50      0      7      0      4      0      0    146    232      0
       0      0      0     96]
 [     0      1     35      0      0      0      0    113    170      0
       0      0      0     40]
 [    59      0      1      0     

In [68]:
confusion_matrix(y_true=y_test, y_pred=y_pred)

array([[   556,      0,      0,      0,      0,      0,      0,      2,
           193,      3,      4,      1,      0,    251],
       [     0,   1270,     11,     15,      0,      0,      0,     56,
           339,      6,      0,      0,      0,   2142],
       [     0,     10,   1875,      2,      0,      1,      0,     78,
           472,      2,      0,      0,      0,      6],
       [     0,     18,     16,    138,      0,      0,      0,      0,
            23,      3,      0,      0,      0,     60],
       [     1,      1,      0,      0,     15,      0,      0,     65,
           363,      0,      0,      0,      0,     90],
       [     0,      0,      0,      1,      0,     38,      0,     61,
           253,      2,      0,      0,      0,      4],
       [     5,      0,      0,      0,      0,      1,      0,      1,
            97,      0,      2,      0,      0,      1],
       [    14,     16,     15,     20,      0,      2,      0,    584,
          2495,     40,  

In [ ]:
# start = time.time()
# seed = 125
# test_size = 0.3
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed)

# fit model no training data
# model = XGBClassifier()
# model.fit(X_train, y_train)
# end = time.time()
# print 'Training in %.2f'%(end-start)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
print(classification_report(y_pred,y_test))

In [ ]:
print(classification_report(y_pred,y_test))